In [1]:
import os
path_now = os.getcwd()

In [2]:
path_data = path_now + r'\train.json'

In [3]:
import json
with open(path_now +r'/'+ r'train.json') as json_data:
    data= json.load(json_data)

In [4]:
listofvariables=list(data[0].keys())

In [5]:
listofvariablestodrop=['giver_username_if_known','number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval',\
                      'request_id','request_text','']

In [6]:
index_of_edited=[]
for index, request in enumerate(data):
    if(request['request_text'] != request["request_text_edit_aware"]):
        index_of_edited.append(index)

number_of_up_minus_down_of_request_at_retrieval = []
for request in data:
        number_of_up_minus_down_of_request_at_retrieval.append(request['number_of_upvotes_of_request_at_retrieval'] - \
        request['number_of_downvotes_of_request_at_retrieval'])

request_id = []
for request in data:
        request_id.append(request['request_id'])

subreddits=[]
for req in data:
    subreddits+=req['requester_subreddits_at_request']


len(set(subreddits))

len(set(request_id))

4040

In [235]:
import pandas as pd

In [236]:
df_roap = pd.read_json(path_data, orient="records")
df_roap.drop(['giver_username_if_known', 'request_text' , 'request_text_edit_aware' , 'requester_username' , 'unix_timestamp_of_request' , 'unix_timestamp_of_request_utc'] , axis=1, inplace=True)

previous_names = list(df_roap.columns)
len(previous_names)

new_names = ["no_of_down_of_request_at_TRI",
"no_of_up_of_request_at_TRI",
"post_edited",
"request_id",
"comments_on_post_at_TRI",
"post_title",
"account_age_at_Q",
"account_age_at_TRI",
"days_since_FP_on_raop_at_Q",
"days_since_FP_on_raop_at_TRI",
"no_of_comments_on_R_at_Q",
"no_of_comments_on_R_at_TRI",
"no_of_comments_on_ROAP_at_Q",
"no_of_comments_on_ROAP_at_TRI",
"no_of_posts_on_R_at_Q",
"no_of_posts_on_R_at_TRI",
"no_of_posts_on_ROAP_at_Q",
"no_of_posts_on_ROAP_at_TRI",
"no_of_subreddits_at_Q",
"success",
"subreddits_at_Q",
"up_minus_down_at_Q",
"up_minus_down_at_TRI",
"up_plus_down_at_Q",
"up_plus_down_at_TRI",
"user_flair"]

dict_name_change = dict(zip(previous_names, new_names))

In [237]:
df_roap.rename(columns = dict_name_change, inplace=True)

In [238]:
df_roap.drop(['request_id'], axis= 1, inplace=True)

In [305]:
df_roap['no_of_up_minus_down_of_request_at_TRI'] = df_roap['no_of_up_of_request_at_TRI'] - df_roap['no_of_down_of_request_at_TRI']

In [306]:
df_roap.drop(['no_of_down_of_request_at_TRI','no_of_up_of_request_at_TRI'], axis =1, inplace = True)

In [ ]:
df.info()

## Crosstab vs Pivot Table 

In [311]:
no_of_unique_values={}
for col_name in df_roap.columns:
    no_of_unique_values.update( {col_name : len(list(df_roap[col_name].unique())) } )

In [ ]:
# Deviation cross tab trial, funtion of 'pandas' not a 'Dataframe'
pd.crosstab(df_roap.no_of_comments_on_ROAP_at_Q, df_roap.success)

In [265]:
import numpy as np

In [ ]:
df_roap.pivot_table(values='account_age_at_Q' , index = 'no_of_comments_on_ROAP_at_Q', columns =['success','user_flair']  , aggfunc =np.mean )

### Using subreddit suscribe(text)

In [275]:
df_roap.subreddits_at_Q= df_roap.subreddits_at_Q.map(lambda x : " ".join(x)) # continue count vectoriser

In [276]:
df_roap.subreddits_at_Q[3] 

'AskReddit DJs IAmA Random_Acts_Of_Pizza'

In [277]:
list_of_subreddits = list(df_roap.subreddits_at_Q) 
list_of_success = list(df_roap.success)

In [279]:
import sklearn

In [280]:
from sklearn import feature_extraction

In [281]:
vectorizer = feature_extraction.text.CountVectorizer( analyzer='word', binary = True, max_features = 10000 )

In [282]:
count_vector = vectorizer.fit_transform(list_of_subreddits)

In [283]:
count_vector = count_vector.toarray()

In [284]:
len(count_vector[1])

7484

### Creating the vector for the succeful people 

In [285]:
count = 0
succesful_people_follow = np.zeros(len(count_vector[1]))
for index, vec in enumerate(count_vector):
    if list_of_success[index] == True:
        succesful_people_follow+= vec
        count+=1
        
succesful_people_follow/=count

In [286]:
from sklearn import metrics

In [287]:
succesful_people_follow = succesful_people_follow.reshape((1,-1))

In [288]:
#Cosine similarity
similarity_list = []
for vec in count_vector:
    similarity_list.append(sklearn.metrics.pairwise.cosine_similarity(vec.reshape((1,-1)), succesful_people_follow))

similarity_list = [x.reshape(1,)[0] for x in similarity_list ]

In [289]:
df_roap.subreddits_at_Q = similarity_list

### Onto next feature

In [270]:
df_roap.user_flair = df_roap.user_flair.map( lambda x : 'Not given' if x is None else x)

In [272]:
df_roap.user_flair.unique()

array(['Not given', 'shroom', 'PIF'], dtype=object)

In [291]:
df_roap.drop('post_title', axis = 1 , inplace = True) # Could be used with handling other text feature

### Preparing data for sklearn 

In [315]:
df_roap.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4040 entries, 0 to 4039
Data columns (total 23 columns):
post_edited                              4040 non-null int64
comments_on_post_at_TRI                  4040 non-null int64
account_age_at_Q                         4040 non-null float64
account_age_at_TRI                       4040 non-null float64
days_since_FP_on_raop_at_Q               4040 non-null float64
days_since_FP_on_raop_at_TRI             4040 non-null float64
no_of_comments_on_R_at_Q                 4040 non-null int64
no_of_comments_on_R_at_TRI               4040 non-null int64
no_of_comments_on_ROAP_at_Q              4040 non-null int64
no_of_comments_on_ROAP_at_TRI            4040 non-null int64
no_of_posts_on_R_at_Q                    4040 non-null int64
no_of_posts_on_R_at_TRI                  4040 non-null int64
no_of_posts_on_ROAP_at_Q                 4040 non-null int64
no_of_posts_on_ROAP_at_TRI               4040 non-null int64
no_of_subreddits_at_Q          

In [318]:
df_roap.corr()

,post_edited,comments_on_post_at_TRI,account_age_at_Q,account_age_at_TRI,days_since_FP_on_raop_at_Q,days_since_FP_on_raop_at_TRI,no_of_comments_on_R_at_Q,no_of_comments_on_R_at_TRI,no_of_comments_on_ROAP_at_Q,no_of_comments_on_ROAP_at_TRI,...,no_of_posts_on_ROAP_at_Q,no_of_posts_on_ROAP_at_TRI,no_of_subreddits_at_Q,success,subreddits_at_Q,up_minus_down_at_Q,up_minus_down_at_TRI,up_plus_down_at_Q,up_plus_down_at_TRI,no_of_up_minus_down_of_request_at_TRI
post_edited,1.000000,0.055510,0.105622,-0.063786,0.058703,-0.183983,0.095751,0.043786,0.027794,0.021147,...,0.008705,0.027325,0.099079,0.049480,0.014956,0.055161,0.031353,0.026695,0.018226,0.048296
comments_on_post_at_TRI,0.055510,1.000000,-0.033698,0.119833,-0.027884,0.177136,-0.020798,0.033959,-0.005649,0.213236,...,-0.015263,0.084098,-0.042057,0.289153,-0.037935,-0.000343,0.039790,-0.002081,0.011177,0.457339
account_age_at_Q,0.105622,-0.033698,1.000000,0.643411,0.231437,-0.270898,0.488341,0.374991,0.054243,0.004942,...,0.033078,0.024190,0.588118,0.041784,0.457273,0.271963,0.217639,0.115681,0.098540,0.047193
account_age_at_TRI,-0.063786,0.119833,0.643411,1.000000,0.071487,0.535403,0.197851,0.315744,0.010430,0.044985,...,-0.048902,-0.013842,0.264835,0.120919,0.271540,0.104312,0.175654,0.032708,0.062263,0.057828
days_since_FP_on_raop_at_Q,0.058703,-0.027884,0.231437,0.071487,1.000000,0.091062,0.204728,0.080732,0.354446,0.198778,...,0.292962,0.158852,0.193807,0.098225,0.139696,0.112943,0.057142,0.078312,0.065913,0.003275
days_since_FP_on_raop_at_TRI,-0.183983,0.177136,-0.270898,0.535403,0.091062,1.000000,-0.252681,-0.011692,0.045414,0.103517,...,-0.020592,0.000806,-0.284938,0.129283,-0.142216,-0.148372,-0.014714,-0.069558,-0.017096,0.009026
no_of_comments_on_R_at_Q,0.095751,-0.020798,0.488341,0.197851,0.204728,-0.252681,1.000000,0.571754,0.122674,0.090045,...,0.068309,0.046250,0.755857,0.033779,0.356694,0.400108,0.245160,0.162262,0.110276,0.034796
no_of_comments_on_R_at_TRI,0.043786,0.033959,0.374991,0.315744,0.080732,-0.011692,0.571754,1.000000,0.055641,0.060981,...,0.035984,0.082653,0.601558,0.105217,0.352549,0.312976,0.527073,0.139605,0.219838,0.089478
no_of_comments_on_ROAP_at_Q,0.027794,-0.005649,0.054243,0.010430,0.354446,0.045414,0.122674,0.055641,1.000000,0.673395,...,0.487150,0.306280,0.082610,0.131296,0.122752,0.129595,0.098129,0.131645,0.129125,-0.000609
no_of_comments_on_ROAP_at_TRI,0.021147,0.213236,0.004942,0.044985,0.198778,0.103517,0.090045,0.060981,0.673395,1.000000,...,0.333418,0.388639,0.045890,0.274253,0.103050,0.071156,0.057434,0.070364,0.068958,0.064886


In [330]:
df_roap_num = df_roap[['post_edited', 'comments_on_post_at_TRI', 'account_age_at_Q',
       'account_age_at_TRI', 'days_since_FP_on_raop_at_Q',
       'days_since_FP_on_raop_at_TRI', 'no_of_comments_on_R_at_Q',
       'no_of_comments_on_R_at_TRI', 'no_of_comments_on_ROAP_at_Q',
       'no_of_comments_on_ROAP_at_TRI', 'no_of_posts_on_R_at_Q',
       'no_of_posts_on_R_at_TRI', 'no_of_posts_on_ROAP_at_Q',
       'no_of_posts_on_ROAP_at_TRI', 'no_of_subreddits_at_Q',
       'subreddits_at_Q', 'up_minus_down_at_Q', 'up_minus_down_at_TRI',
       'up_plus_down_at_Q', 'up_plus_down_at_TRI',
       'no_of_up_minus_down_of_request_at_TRI']]

In [331]:
df_roap_non_num = df_roap[['success', 'user_flair']]

In [332]:
df_roap_non_num.columns

Index(['success', 'user_flair'], dtype='object')

In [342]:
np_values = df_roap_num.values[:,].astype('float64')

In [343]:
df_roap_y = df_roap['success']

In [347]:
# Dont run it again, as df is now changed
df_roap_y = df_roap_y.map({False : 0 , True : 1 }) 

### Stroing Pickle files 

In [349]:
import pickle

In [351]:
with open(path_now + '\\feature_df.p' ,'wb') as filetowriteon:
    pickle.dump(df_roap_num, filetowriteon)
    filetowriteon.close()

In [352]:
with open(path_now + '\\to_predict.p' ,'wb') as filetowriteon:
    pickle.dump(df_roap_y, filetowriteon)
    filetowriteon.close()